In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

# import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchmetrics as TM
# pl.utilities.seed.seed_everything(seed=42)
import numpy as np
import pandas as pd

import sys, os
source_path = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'metrics')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'predict')
sys.path.append(source_path)
import matplotlib.pyplot as plt

from dl import NeuralNetwork, Trainer
from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    get_predict_loader,
    ts_split,
    cont_cat_split,
    dataloader_by_stock,
    get_data,
    dataloader_test_by_stock
)
from metrics import calc_spread_return_sharpe
from predict import run_pred_step
print(torch.__version__)

import pickle

import warnings
warnings.filterwarnings("ignore")

C:\Users\gilbe\anaconda3\envs\tokyo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.11.0


# Get data

In [2]:
FOLDER = 'example_test_files'
df = get_data(folder=FOLDER)
# df['Target'] = df['Close'].pct_change()
# df = df.sort_values(by=['RowId', 'Date'])
print(df.head())

"""
SUBTRACT:
*) 3 FROM CONTINUOUS COLS BECAUSE OF POOLING
*) 1 FROM CAT_FEATURES TO MAKE MATRICES MATCH AFTER TORCH.CAT
"""
CONT_COLS=['Close', 'Open', 'High', 'Low', 'MarketCapitalization',         
           # 'NetSales', 'EquityToAssetRatio', 'TotalAssets', 'Profit', 
           # 'OperatingProfit', 'EarningsPerShare', 'Equity', 
           # 'BookValuePerShare', 'ResultDividendPerShare1stQuarter', 
           # 'ResultDividendPerShare2ndQuarter', 'ResultDividendPerShare3rdQuarter',
           # 'ResultDividendPerShareFiscalYearEnd', 'ResultDividendPerShareAnnual'
          ]
TS_IN_FEATURES = len(CONT_COLS)
CAT_FEATURES = 4 + 4 - 1 # TEXT_COLS = ['Section/Products', '33SectorName', '17SectorName', 'Universe0']
print('CAT_FEATURES:', CAT_FEATURES)
EMBEDDING_DIM = 300
NO_EMBEDDING = 2000 #2 * len(df_train_cat)

train_df.head(10):
            Section/Products  33SectorName  17SectorName  Universe0  \
Date                                                                  
2021-12-06                 0             6             8          0   
2021-12-07                 0             6             8          0   
2021-12-06                 0             6             8          0   
2021-12-07                 0             6             8          0   
2021-12-06                 0             6             8          0   
2021-12-07                 0             6             8          0   
2021-12-06                 0             6             8          0   
2021-12-07                 0             6             8          0   
2021-12-06                 0             6             8          0   
2021-12-07                 0             6             8          0   

            MarketCapitalization  SecuritiesCode          RowId    Open  \
Date                                                 

In [3]:
df['SecuritiesCode'].unique()

array([1301, 1332, 1333, ..., 9993, 9994, 9997], dtype=int64)

In [4]:
df[df['SecuritiesCode'] == 1301]

,Section/Products,33SectorName,17SectorName,Universe0,MarketCapitalization,SecuritiesCode,RowId,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
Date,,,,,,,,,,,,,,,
2021-12-06,0,6,8,0,3.365911e+10,1301,20211206_1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False
2021-12-07,0,6,8,0,3.365911e+10,1301,20211207_1301,2998.0,3065.0,2990.0,3065.0,19100,1.0,NaN,False


In [5]:
df[df['SecuritiesCode'] == 1301].shape

(2, 15)

In [8]:
stocks = df['SecuritiesCode'].unique()
count = 0
BATCH_SIZE = 512
weight_decay = 0.01

# model = create_model()
# trainer = Trainer(model, optimizer_name='rmsprop', lr=1.3333e-4, weight_decay=weight_decay)
# torch.autograd.set_detect_anomaly(True)
trained_model = torch.load('./trained_model.pt')
with open('./scaler_dict.pkl', 'rb') as f:
    scaler_dict = pickle.load(f)

    
pred_df = pd.DataFrame()

for stock in stocks:
    teast_dataloader = None
    if count > 500:
        break
    try:
        print(f'Run prediction for stock: {stock}')
        transformer = scaler_dict[stock]
        test_dataloader = dataloader_test_by_stock(
            df, 
            stock, 
            transformer=transformer,
            batch_size=BATCH_SIZE, 
            continous_cols=CONT_COLS,
            target_col=None
        )
        pred_list = run_pred_step(test_dataloader, trained_model, x_cat=True, target=None)
        df_pred = pd.DataFrame(pred_list, index=[stock])
        pred_df = pd.concat([pred_df, df_pred])#.pivot()
        print('pred_df')
        print(pred_df)
        # pred_df[stock] = pred_list
        print()
        count += 1
    except Exception as e:
        print(f'Exception {e}')
        

# pred_df = pd.DataFrame.from_dict(pred_df)

    

Run prediction for stock: 1301
Test predictions:
[-0.01412284 -0.02129873] (2,)
pred_df
             0         1
1301 -0.014123 -0.021299

Run prediction for stock: 1332
Test predictions:
[-0.01677142 -0.00093737] (2,)
pred_df
             0         1
1301 -0.014123 -0.021299
1332 -0.016771 -0.000937

Run prediction for stock: 1333
Test predictions:
[-0.00691961 -0.02188414] (2,)
pred_df
             0         1
1301 -0.014123 -0.021299
1332 -0.016771 -0.000937
1333 -0.006920 -0.021884

Run prediction for stock: 1375
Test predictions:
[ 0.00479659 -0.02254717] (2,)
pred_df
             0         1
1301 -0.014123 -0.021299
1332 -0.016771 -0.000937
1333 -0.006920 -0.021884
1375  0.004797 -0.022547

Run prediction for stock: 1376
Test predictions:
[-0.00745573 -0.026214  ] (2,)
pred_df
             0         1
1301 -0.014123 -0.021299
1332 -0.016771 -0.000937
1333 -0.006920 -0.021884
1375  0.004797 -0.022547
1376 -0.007456 -0.026214

Run prediction for stock: 1377
Test predictions:
[-0.00

In [15]:
pred_df.T

,1301,1332,1333,1375,1376,1377,1379,1381,1407,1413,...,3865,3880,3891,3900,3901,3902,3903,3906,3914,3915
0,-0.014123,-0.016771,-0.006920,0.004797,-0.007456,-0.001582,-0.006810,-0.007446,0.007330,-0.014953,...,0.006186,0.006190,0.000444,-0.015894,0.005743,-0.005670,-0.010369,-0.016106,0.000588,-0.001203
1,-0.021299,-0.000937,-0.021884,-0.022547,-0.026214,-0.022368,-0.023924,-0.023110,-0.021133,-0.014566,...,-0.020018,-0.025269,-0.023124,-0.019540,-0.008894,-0.009896,-0.010209,-0.020645,-0.019928,-0.009493


# Load model